In [1]:
# Load the numpy, pandas, matplotlib, and os libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from sklearn.linear_model import LinearRegression # linear regression
import statsmodels.formula.api as smf # regression modeling

In [2]:
# Set the working directory to your project folder.
os.chdir('C:/Users/ual-laptop/Desktop/Mac/Fall semester/MIS 545/Group Project/V2')

In [3]:
# Read CollegeTuition.csv into a data frame called collegeTuition.
collegeTuition = pd.read_csv('CollegeTuition.csv')


In [4]:
collegeTuition = collegeTuition.rename(columns={
    'Institution Type': 'Institution_Type',
    'School Type': 'School_Type',
    'Total (Inflation Adjusted)': 'Total_Inflation_Adjusted',
    'Tuition & Fees (Inflation Adjusted)': 'Tuition_Fees_Inflation_Adjusted',
    'Room (Inflation Adjusted)': 'Room_Inflation_Adjusted',
    'Board (Inflation Adjusted)': 'Board_Inflation_Adjusted',
    'Tuition & Fees' : 'Tuition_Fees',
})

# Verify the changes by printing the updated column names
print(collegeTuition.columns)


Index(['Institution_Type', 'Year', 'School_Type', 'Total_Inflation_Adjusted',
       'Tuition_Fees_Inflation_Adjusted', 'Room_Inflation_Adjusted',
       'Board_Inflation_Adjusted', 'Total', 'Tuition_Fees', 'Room', 'Board'],
      dtype='object')


In [5]:
# Check the data type of the 'Year' column
print(collegeTuition['Year'].dtype)

# If 'Year' is not a string type, convert it to string first
collegeTuition['Year'] = collegeTuition['Year'].astype(str)

# Then split by hyphen and convert to integer
collegeTuition['Year'] = collegeTuition['Year'].str.split('-').str[0].astype(int)
# This code first converts the 'Year' column to a string type, then splits it at the hyphen, and takes the first part of the split

object


In [7]:
#print the first 5 rows of the data frame
print(collegeTuition.head())

   Institution_Type  Year                      School_Type  \
0  All Institutions  1966  All (4 and 2 Year Institutions)   
1  All Institutions  1967  All (4 and 2 Year Institutions)   
2  All Institutions  1968  All (4 and 2 Year Institutions)   
3  All Institutions  1969  All (4 and 2 Year Institutions)   
4  All Institutions  1970  All (4 and 2 Year Institutions)   

   Total_Inflation_Adjusted  Tuition_Fees_Inflation_Adjusted  \
0              10387.488583                      4326.573312   
1              10324.923309                      4286.652333   
2              10147.168479                      4148.525498   
3              10247.153284                      4237.619973   
4              10322.808184                      4297.450799   

   Room_Inflation_Adjusted  Board_Inflation_Adjusted        Total  \
0              2476.106063               3584.809208  1378.029556   
1              2495.184685               3543.086290  1415.127765   
2              2506.262771         

In [6]:
# One-Hot Encode the 'Institution Type'
# Use pandas get_dummies method for one-hot encoding
institution_type_dummies = pd.get_dummies(collegeTuition['Institution_Type'])

# Explanation
# pd.get_dummies(collegeTuition['Institution_Type']): Performs one-hot encoding on 'Institution_Type'
# institution_type_dummies: Stores the result of one-hot encoding into institution_type_dummies variable

In [7]:
# Merge the original data with the one-hot encoded data
collegeTuition_processed = pd.concat([collegeTuition, institution_type_dummies], axis=1)

In [8]:
# Correctly rename the columns by replacing spaces with underscores
collegeTuition_processed.columns = collegeTuition_processed.columns.str.replace(' ', '_')

# Confirm the changes
print(collegeTuition_processed.columns)

Index(['Institution_Type', 'Year', 'School_Type', 'Total_Inflation_Adjusted',
       'Tuition_Fees_Inflation_Adjusted', 'Room_Inflation_Adjusted',
       'Board_Inflation_Adjusted', 'Total', 'Tuition_Fees', 'Room', 'Board',
       'All_Institutions', 'Private_Institutions', 'Public_Institutions'],
      dtype='object')


In [9]:
# Generate the linear regression model and save it in an object called collegeTuitionModel.

import statsmodels.formula.api as smf

# Generate the linear regression model for Hypothesis 1
collegeTuitionModel = smf.ols(
    formula='Tuition_Fees_Inflation_Adjusted ~ Year + Private_Institutions + Public_Institutions', 
    data=collegeTuition_processed
).fit()

# Here, I've replaced 'Tuition & Fees (Inflation Adjusted)' with 'Tuition_and_Fees_Inflation_Adjusted'
# because it seems that the column names have had their spaces removed. 
# Make sure to replace this with the exact column name from your DataFrame.


In [10]:
# Display the beta coefficients for the model.
collegeTuitionModel.params

Intercept                      -402618.039283
Private_Institutions[T.True]      9343.003430
Public_Institutions[T.True]      -2738.540092
Year                               205.222719
dtype: float64

In [13]:
# Display the linear regression model results summary. (I used statsmodels’ .ols(), .fit(), and .summary(), but there are other ways to do it too.)
collegeTuitionModel.summary()


<class 'statsmodels.iolib.summary.Summary'>
"""
                                   OLS Regression Results                                  
===========================================================================================
Dep. Variable:     Tuition_Fees_Inflation_Adjusted   R-squared:                       0.756
Model:                                         OLS   Adj. R-squared:                  0.755
Method:                              Least Squares   F-statistic:                     480.2
Date:                             Wed, 22 Nov 2023   Prob (F-statistic):          7.85e-142
Time:                                     11:08:48   Log-Likelihood:                -4471.9
No. Observations:                              468   AIC:                             8952.
Df Residuals:                                  464   BIC:                             8968.
Df Model:                                        3                                         
Covariance Type:                         nonrobust                                         
================================================================================================
                                   coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------
Intercept                    -4.026e+05    2.1e+04    -19.129      0.000   -4.44e+05   -3.61e+05
Private_Institutions[T.True]  9343.0034    388.495     24.049      0.000    8579.575    1.01e+04
Public_Institutions[T.True]  -2738.5401    388.495     -7.049      0.000   -3501.968   -1975.112
Year                           205.2227     10.568     19.420      0.000     184.456     225.989
==============================================================================
Omnibus:                        1.268   Durbin-Watson:                   0.052
Prob(Omnibus):                  0.531   Jarque-Bera (JB):                1.324
Skew:                           0.078   Prob(JB):                        0.516
Kurtosis:                       2.791   Cond. No.                     2.64e+05
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.64e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [11]:
# Test for multicollinearity using the variance inflation factor (VIF) and display the results.
from statsmodels.stats.outliers_influence import variance_inflation_factor
variables = collegeTuitionModel.model.exog
vif = [variance_inflation_factor(variables, i) for i in range(variables.shape[1])]
vif

[17610.42397336939, 1.3333333333333337, 1.3333333333333335, 1.0]